In [1]:
import pygrib
grbs = pygrib.open('sampledata/flux.grb')  
grbs.seek(2)
grbs.tell()

2

### pygrib open instances behave like regular python file objects, with seek, tell, read, readline and close methods, except that offsets are measured in grib messages instead of bytes: 

In [2]:
grb = grbs.read(1)[0] # read returns a list with the next N (N=1 in this case) messages.
grb # printing a grib message object displays summary info

3:Maximum temperature:K (instant):regular_gg:heightAboveGround:level 2 m:fcst time 108-120 hrs:from 200402291200

In [3]:
grbs.tell()

3

### print an inventory of the file: 

In [4]:
grbs.seek(0)
for grb in grbs:
    print(grb)

1:Precipitation rate:kg m**-2 s**-1 (avg):regular_gg:surface:level 0:fcst time 108-120 hrs (avg):from 200402291200
2:Surface pressure:Pa (instant):regular_gg:surface:level 0:fcst time 120 hrs:from 200402291200
3:Maximum temperature:K (instant):regular_gg:heightAboveGround:level 2 m:fcst time 108-120 hrs:from 200402291200
4:Minimum temperature:K (instant):regular_gg:heightAboveGround:level 2 m:fcst time 108-120 hrs:from 200402291200


### find the first grib message with a matching name: 

In [5]:
grb = grbs.select(name='Maximum temperature')[0]
grb

3:Maximum temperature:K (instant):regular_gg:heightAboveGround:level 2 m:fcst time 108-120 hrs:from 200402291200

### extract the data values using the 'values' key (grb.keys() will return a list of the available keys): 

In [6]:
# The data is returned as a numpy array, or if missing values or a bitmap
# are present, a numpy masked array.  Reduced lat/lon or gaussian grid
# data is automatically expanded to a regular grid. Details of the internal
# representation of the grib data (such as the scanning mode) are handled
# automatically.
maxt = grb.values # same as grb['values']
print(maxt.shape,"\n", maxt.min(),"\n" ,maxt.max(),"\n")

(94, 192) 
 223.70000000000002 
 319.90000000000003 



### get the latitudes and longitudes of the grid: 

In [7]:
lats, lons = grb.latlons()
print(lats.shape,"\n", lats.min(),"\n", lats.max(),"\n", lons.shape,"\n",lons.min(),"\n", lons.max(),"\n")

(94, 192) 
 -88.54195013729753 
 88.54195013729753 
 (94, 192) 
 0.0 
 358.125 



### get the second grib message: 

In [8]:
grb = grbs.message(2) # same as grbs.seek(1); grb=grbs.readline()
print(grb)
#print(dir(grb))

2:Surface pressure:Pa (instant):regular_gg:surface:level 0:fcst time 120 hrs:from 200402291200


### modify the values associated with existing keys (either via attribute or dictionary access): 

In [9]:
grb['forecastTime'] = 240
grb.dataDate = 20100101

### get the binary string associated with the coded message: 

In [10]:
msg = grb.tostring()
grbs.close() # close the grib file.

### write the modified message to a new GRIB file: 

In [11]:
grbout = open('test.grb','wb')
grbout.write(msg)
grbout.close()
pygrib.open('test.grb').readline() 

1:Surface pressure:Pa (instant):regular_gg:surface:level 0:fcst time 240 hrs:from 201001011200